In [2]:
import os
import xarray as xr
import numpy as np
import torch
from tqdm import tqdm
import time

# ====== ⚙️ Settings ⚙️ ===============================
days = 100                                  # 학습할 데이터 일수
gap_min = 1                                 # 데이터 간격 (분)
delta_min = 30                              # 몇분 뒤를 예측할 것인지
start_idx = 576000                          # 시작 타임 인덱스 (576000은 40일 후의 인덱스)
end_idx = start_idx + days * 24 * 60 * 10   # 실제 마지막 타임 인덱스

var_name = 'PWUV'
var_list = ['PW', 'USFC', 'VSFC', 'PSFC']  # 사용할 변수 목록

base_dir = "/scratch/x3108a06/input_data/2D/"
input_dir = f'{base_dir}{var_name}_{gap_min}m'
pad_dir = f"{base_dir}{var_name}_d{delta_min}m_{gap_min}m"
# ========================================================

In [2]:
# =======================
#      .nc to .pt
# =======================


os.makedirs(input_dir, exist_ok=True)


# if not os.path.exists(os.path.join(input_dir, f"mean_std_{days}d_{gap_min}m.npy")):
mean_array = np.zeros((len(var_list),))
mean2_array = np.zeros((len(var_list),))
count = 0

for i in tqdm(range(start_idx, start_idx + (days * 24 * 60 * 10) * 7 //10 + 1, gap_min * 100), desc="Preprocessing Data"):
    with xr.open_dataset(
        f"/scratch/x3108a06/nc_data/OUT_2D_nc/rcemip_small_awing_96x96x74_1km_6s_300K_48_{i:010d}.2Dcom_1.nc",
        engine="netcdf4"
    ) as ds:

        ds_sliced = ds[var_list].isel(time=0, drop=True)

        data_tensor = torch.stack([
            torch.tensor(ds_sliced[var].values, dtype=torch.float32)
            for var in var_list
        ])  # (C=2, H, W)

        # torch.save(data_tensor, os.path.join(input_dir, f"{i:010d}.pt"))

        # torch.Tensor → numpy 전환 후 평균 계산
        data_np = data_tensor.numpy()
        z_mean = data_np.mean(axis=(1, 2))      # (C,)
        z2_mean = (data_np**2).mean(axis=(1, 2))

        mean_array += z_mean
        mean2_array += z2_mean
        count += 1

mean = mean_array / count
mean2 = mean2_array / count
std = np.sqrt(mean2 - mean**2)

np.save(os.path.join(input_dir, f"mean_std_{days}d_{gap_min}m.npy"), np.array([mean, std]))


# =======================
#        padding
# =======================


def cyclic_pad_2d(x: torch.Tensor, pad: int) -> torch.Tensor:
    x = torch.cat([x[..., -pad:, :], x, x[..., :pad, :]], dim=-2)
    x = torch.cat([x[..., :, -pad:], x, x[..., :, :pad]], dim=-1)
    return x

for i in tqdm(range(start_idx, start_idx + days * 24 * 60 * 10 + 1, gap_min * 10), desc="Preprocessing Data"):
    with xr.open_dataset(
        f"/scratch/x3108a06/nc_data/OUT_2D_nc/rcemip_small_awing_96x96x74_1km_6s_300K_48_{i:010d}.2Dcom_1.nc",
        engine="netcdf4"
    ) as ds:

        ds_sliced = ds[var_list].isel(time=0, drop=True)

        data_tensor = torch.stack([
            cyclic_pad_2d(
                (torch.tensor(ds_sliced[var].values, dtype=torch.float32) - mean[j]) / std[j],
                16
            )
            for j, var in enumerate(var_list)
        ])  # (C, H, W)

        torch.save(data_tensor, os.path.join(input_dir, f"{i:010d}.pt"))

Preprocessing Data: 100%|██████████| 144001/144001 [1:35:46<00:00, 25.06it/s]  


In [2]:
# =======================
#      (x, y) stack
# =======================

# x, y를 첫번째 차원으로 쌓아서 (2, C, H, W) 모양의 텐서를 만드는 코드

os.makedirs(pad_dir, exist_ok=True)

# 파일 리스트 정렬
filelist = sorted([f for f in os.listdir(input_dir) if f.endswith('.pt')])
N = len(filelist)

# 몇번째 뒤와 x, y쌍으로 묶을 것인지
shift = delta_min // gap_min

for idx, fname in tqdm(enumerate(filelist)):  # type: int, str
    # 최종 결과 파일의 경로를 먼저 확인합니다.
    output_fpath = os.path.join(pad_dir, fname)

    # 만약 결과 파일이 이미 존재한다면, 이번 순서는 건너뜁니다.
    if os.path.exists(output_fpath):
        continue  # 다음 파일로 바로 넘어감

    fpath = os.path.join(input_dir, fname)
    data = torch.load(fpath, map_location='cpu')
    if not isinstance(data, torch.Tensor):
        print(f"{fname}: Not a tensor, skipped.")
        continue

    # 현재 파일
    # padded_0 = cyclic_pad_2d(data, 16)

    # shift만큼 뒤 파일(없으면 마지막 파일 사용)
    shifted_idx = min(idx + shift, N - 1)
    shifted_fname = filelist[shifted_idx]
    shifted_fpath = os.path.join(input_dir, shifted_fname)
    shifted_data = torch.load(shifted_fpath, map_location='cpu')
    
    # padded_1 = cyclic_pad_2d(shifted_data, 16)

    # (2, 128, 128) 저장
    out = torch.stack([data, shifted_data], dim=0)
    torch.save(out, os.path.join(pad_dir, fname))


144001it [06:26, 372.26it/s] 


In [6]:
# =======================
#    train test split
# =======================

import shutil
from math import floor


# 앞에서부터 70%를 train으로, 15%를 valid로, 15%를 test 셋으로 분리하여 저장하는 코드

def split_and_copy(src_dir, dst_base, prefix='', total_limit=None):
    os.makedirs(dst_base, exist_ok=True)
    files = sorted([f for f in os.listdir(src_dir) if os.path.isfile(os.path.join(src_dir, f))])
    if total_limit:
        files = files[:total_limit]
    n = len(files)
    n_train = floor(n * 0.7)
    n_valid = floor(n * 0.15)
    n_test = n - n_train - n_valid

    splits = [
        ('train', n_train),
        ('valid', n_valid),
        ('test', n_test),
    ]

    print(f"[{prefix}] Total: {n} -> train: {n_train}, valid: {n_valid}, test: {n_test}")

    idx = 0
    for split, count in splits:
        dst_dir = os.path.join(dst_base, prefix, split)
        os.makedirs(dst_dir, exist_ok=True)
        for i in tqdm(range(count), desc=split):
            src_path = os.path.join(src_dir, files[idx])
            dst_path = os.path.join(dst_dir, files[idx])
            idx += 1
            if os.path.exists(dst_path):
                continue  # 다음 파일로 바로 넘어감
            shutil.copy2(src_path, dst_path)


# ============== settings ====================
split_and_copy(
    src_dir=pad_dir,
    dst_base=base_dir,
    prefix=f'{var_name}_d{delta_min}m_{gap_min}m',
    total_limit=None
)
# ============================================

[PWUV_d30m_1m] Total: 144001 -> train: 100800, valid: 21600, test: 21601


test: 100%|██████████| 21601/21601 [00:00<00:00, 53621.80it/s]
